In [1]:
#connection to Snowflake account
from snowflake.snowpark import Session

connection_parameters = {
    "account": "<your account>",
    "user": "<your username>",
    "password": "<your password>",
   "role": "SYSADMIN",  # optional
   "warehouse": "COMPUTE_WH",  # optional
    "database": "DATA_ENGINEERING",  # optional
    "schema": "PUBLIC",  # optional
} 

new_session = Session.builder.configs(connection_parameters).create()  

#verify the connection
print(new_session.sql("select current_warehouse(), current_database(), current_schema()").collect()) 



[Row(CURRENT_WAREHOUSE()='COMPUTE_WH', CURRENT_DATABASE()='DATA_ENGINEERING', CURRENT_SCHEMA()='PUBLIC')]


In [8]:
#working with data frames - create a dataFrame which holds our TRIPS table
df_tweets = new_session.table("gpt_tweets")

#showing fist 10 rows
df_tweets.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"                 |"DATE"                     |"USERNAME"  |"TWEET"                                             |"REPLYCOUNT"  |"RETWEETCOUNT"  |"LIKECOUNT"  |"QUOTECOUNT"  |"ONLYDATE"  |"ONLYHOUR"  |"ONLYMIN"  |"PROCESSED_TWEET"                                   |"SENTIMENT_LABEL"  |"SENTIMENT_SCORE"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1638329623946878976  |2023-03-21 23:59:55+00:00  |lqgds36373  |ChatGP

In [3]:
#create dataframe from SQL query
df_sql = new_session.sql("select distinct username from gpt_tweets ")
df_sql.show()

--------------
|"USERNAME"  |
--------------
|lqgds36373  |
|yxwec12342  |
|cwsea23772  |
|jerje51666  |
|wwxly15746  |
|xfcfc69355  |
|cjxpg99081  |
|qpbce71884  |
|vqphq79129  |
|ilsfk08081  |
--------------



In [4]:
#dataset transformations
from snowflake.snowpark.functions import col
df_filtered = new_session.table('gpt_tweets').filter(col("username") == 'lqgds36373')
df_filtered.show()



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"                 |"DATE"                     |"USERNAME"  |"TWEET"                            |"REPLYCOUNT"  |"RETWEETCOUNT"  |"LIKECOUNT"  |"QUOTECOUNT"  |"ONLYDATE"  |"ONLYHOUR"  |"ONLYMIN"  |"PROCESSED_TWEET"                 |"SENTIMENT_LABEL"  |"SENTIMENT_SCORE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1638329623946878976  |2023-03-21 23:59:55+00:00  |lqgds36373  |ChatGPT is another woke machine.   |4             |4               |32           |0             |2023-03-21  |2

In [5]:
#select only some columns from table
df_few_columns = df_filtered.select(col('date'), col('username'),col('tweet'))
df_few_columns.show()

------------------------------------------------------------------------------
|"DATE"                     |"USERNAME"  |"TWEET"                            |
------------------------------------------------------------------------------
|2023-03-21 23:59:55+00:00  |lqgds36373  |ChatGPT is another woke machine.   |
------------------------------------------------------------------------------



In [7]:
#query to find users with at least two tweets and summarize total number of tweets together with tweet's
#statistics

df_summary = new_session.sql("""
select username, count(id) tweets_count, sum(replycount) sum_reply, sum(retweetcount) sum_retweet, sum(likecount) sum_like
from gpt_tweets
group by username
having count(id) > 1
order by count(id) desc
""")

#create a view
df_summary.create_or_replace_view('tweet_active_users_summary')

#query a view
df_view = new_session.table('tweets_active_users_summary')
df_view.show()

--------------------------------------------------------------------------
|"USERNAME"  |"TWEETS_COUNT"  |"SUM_REPLY"  |"SUM_RETWEET"  |"SUM_LIKE"  |
--------------------------------------------------------------------------
|oemar25954  |514             |133          |1              |173         |
|evtpb31968  |373             |1            |0              |2           |
|jrweq99733  |310             |0            |0              |1           |
|cwsea23772  |286             |76           |19             |99          |
|shjkn52473  |238             |0            |0              |0           |
|gakfn99838  |233             |268          |0              |45          |
|vamcz49922  |197             |1            |0              |0           |
|hgznd53107  |182             |1            |0              |0           |
|pmpew21354  |182             |239          |34             |704         |
|zciwy86203  |167             |476          |99             |1308        |
-------------------------

In [22]:
#same output by using data frame functions
from snowflake.snowpark.functions import col, count, avg, date_trunc, round, sum

df_summary2 = df_tweets.group_by('username')\
                        .agg(count('id').alias('tweets_count'), \
                             sum('replycount').alias('sum_replies'), \
                             sum('retweetcount').alias('sum_retweets')) \
                        .select('username', 'tweets_count', 'sum_replies', 'sum_retweets') \
                        .sort(col('tweets_count').desc())\
                        .filter(col('tweets_count')>1)

df_summary2.show()

#create a view
df_summary2.create_or_replace_view('tweets_active_users_summary')

#query a view
df_view = new_session.table('tweets_active_users_summary')
df_view.show()


----------------------------------------------------------------
|"USERNAME"  |"TWEETS_COUNT"  |"SUM_REPLIES"  |"SUM_RETWEETS"  |
----------------------------------------------------------------
|oemar25954  |514             |133            |1               |
|evtpb31968  |373             |1              |0               |
|jrweq99733  |310             |0              |0               |
|cwsea23772  |286             |76             |19              |
|shjkn52473  |238             |0              |0               |
|gakfn99838  |233             |268            |0               |
|vamcz49922  |197             |1              |0               |
|pmpew21354  |182             |239            |34              |
|hgznd53107  |182             |1              |0               |
|zciwy86203  |167             |476            |99              |
----------------------------------------------------------------

----------------------------------------------------------------
|"USERNAME"  |"TWEETS_CO

In [42]:
#try to find how long is the shortest and longest tweets


-------------------------------
|"MIN_LENGTH"  |"MAX_LENGTH"  |
-------------------------------
|5             |332           |
-------------------------------



In [ ]:
#close the session
new_session.close()